# Codes for data extration and transformation

In [49]:
import sqlite3 as sq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
## Function for creating data frame from sqlite database
def make_df(table_name,*columns):
    conn = sq.connect('db.sqlite3')
    sql = 'select '
    sql += ','.join(columns) + ' from ' + table_name
    df = pd.read_sql_query(sql,conn)
    conn.close()
    df.columns = [i for i in columns]
    return df

In [51]:
qid_res = make_df('surveys_response','questionID_id','response')
qid_res.head()

,questionID_id,response
0,16,5
1,17,3
2,18,3
3,19,1
4,20,5


In [52]:
qstn_txt = make_df('surveys_question_text','questionTextID','factorID_id','positive_p')
qstn_txt.shape

(118, 3)

In [53]:
surv = make_df( 'surveys_survey','surveyID', 'userID_id', 'creationDate','completionDate')
surv.head()

,surveyID,userID_id,creationDate,completionDate
0,33,5,2021-03-09 21:07:01,2021-03-10 03:10:08.933258
1,34,6,2021-03-09 21:07:01,2021-03-10 05:28:51.943376
2,35,7,2021-03-09 21:07:02,None
3,36,8,2021-03-09 21:07:02,None
4,37,9,2021-03-09 21:07:02,None


In [54]:
factor = make_df('surveys_factor','factorID', 'factorName', 'studyID_id')
factor.head()

,factorID,factorName,studyID_id
0,1,Factor 1,1
1,2,Factor 2,1
2,3,Factor 3,1
3,4,Factor 4,1
4,5,Factor 5,1


In [55]:
qstn = make_df('surveys_question','questionID' , 'questionTextID_id' , 'surveyID_id')
factor.head()

,factorID,factorName,studyID_id
0,1,Factor 1,1
1,2,Factor 2,1
2,3,Factor 3,1
3,4,Factor 4,1
4,5,Factor 5,1


In [56]:
user = make_df('surveys_user','userID', 'userGroup', 'age', 'location', 'hireDate')
user.head()

,userID,userGroup,age,location,hireDate
0,1,The Boss,NaN,None,None
1,2,test,NaN,None,None
2,3,test,NaN,None,None
3,4,test,NaN,None,None
4,5,Sophomore,NaN,None,None


In [57]:
dw_df = pd.merge(qid_res,qstn,left_on = 'questionID_id',right_on='questionID',how='inner')
dw_df = pd.merge(dw_df,qstn_txt,left_on = 'questionTextID_id',right_on='questionTextID',how='inner')
dw_df = pd.merge(dw_df,factor,left_on = 'factorID_id',right_on='factorID',how='inner')
dw_df = pd.merge(dw_df,surv,left_on = 'surveyID_id',right_on='surveyID',how='inner')
dw_df.shape

(8296, 15)

In [58]:
dw_df.isnull().any()

questionID_id        False
response             False
questionID           False
questionTextID_id    False
surveyID_id          False
questionTextID       False
factorID_id          False
positive_p           False
factorID             False
factorName           False
studyID_id           False
surveyID             False
userID_id            False
creationDate         False
completionDate        True
dtype: bool

In [59]:
response_values = list(pd.unique(dw_df['response']))
string_response = [r for r in response_values if r not in '123456']
string_response

['jh', 'jbj', 'False', 'sd', 'asd']

In [60]:
b = ~dw_df.response.isin(string_response)
dw_df = dw_df[b]
dw_df['response'] = pd.to_numeric(dw_df.response)
dw_df.shape

(8291, 15)

In [61]:
response = dw_df.response
dw_df.loc[dw_df.positive_p==0,'response'] = (7- response)
dw_df['creationDate'] = pd.to_datetime(dw_df['creationDate'])

In [62]:
dw_df.to_csv('dw_df.csv')